In [1]:
from scipy.optimize import linprog
import numpy as np
import cvxpy as cp 
import pandas as pd

$$
\min z = -x_1 + 4_2, \\
s.t. 
\begin{cases}
-3x_1 + x_2 \le 6, \\
x_1 + 2 x_2 \le 4, \\
x_2 \ge -3.
\end{cases}

In [2]:
c = np.array([-1, 4])
A_ub = np.array([[-3, 1], [1, 2]])
b_ub = np.array([6, 4])

bounds = np.array([[None, None], [-3, None]])

res = linprog(c, A_ub, b_ub, A_eq=None, b_eq=None, bounds=bounds)
print("目标函数的最小值：", res.fun)
print("最优解为：", res.x)

目标函数的最小值： -22.0
最优解为： [10. -3.]


$$
\min w = -x_1 + 2x_2 + 3x_3, \\
s. t.
\begin{cases}
-2x_1 + x_2 + x_3 \le 9 \\
3x_1 - x_2 - 2x_3 \le -4 \\
4x_1 - 2x_2 - x_3 = -6 \\
x_1 \ge -10,\ x2 \ge 0
\end{cases}

In [4]:
c = [-1, 2, 3]
A_ub = [[-2, 1, 1], [3, -1, -2]]
b_ub = [[9], [-4]]
A_eq = [[4, -2, -1]]
b_eq = [-6]
bounds = [[-10, None], [0, None], [None, None]]
res = linprog(c, A_ub, b_ub, A_eq, b_eq, bounds)
print("目标函数的最小值：", res.fun)
print("最优解为：", res.x)

目标函数的最小值： 0.399999999999999
最优解为： [-1.6  0.  -0.4]
